In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"]=SERPER_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [ ]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": "2002년 월드컵 4강 국가 알려줘"
    }]
)

In [ ]:
from langchain_openai import ChatOpenAI
chat=ChatOpenAI(
    model_name='gpt-4o-mini'
)
chat.invoke("안녕~ 너를 소개해 줄래?")

In [ ]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        """
        너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}개 추천하고,
        그 요리의 레시피를 제시해 줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
    )
)
prompt

In [ ]:
prompt.invoke({"개수": 2, "재료": "사과, 잼"})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}"),
        ("human", "Hello, how atr you doing?"),
        ("ai", "I'm doing well, thinks!"),
        ("human", "{user_input}")
    ]
)

message = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(message)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(model="gpt-4o-mini")

chain=prompt|model|StrOutputParser()
chain.invoke({"topic": "ice cream"})

In [ ]:
model = ChatOpenAI(model="gpt-4o")
prompt=ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chain = prompt|model

for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm=ChatOpenAI(model="gpt-4o-mini", temperature=0)

chat_template=ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘. ex. Query: SF 영화 3개를 추천해 줘/ 답변: ['인터스텔라', '스페이스오디세이','혹성탈출']")
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)
chain=chat_template|llm
chain.invoke("액션")

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt=PromptTemplate(
    template="List {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = ChatOpenAI(model="gpt-4o-mini")

chain=prompt|model|output_parser
chain.invoke({"subject": "공포 영화"})

In [ ]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")

parser=JsonOutputParser(pydantic_object=Country)
prompt=PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain=prompt|model|parser

country_query = "아르헨티나는 어떤 나라야?"
chain.invoke({"query": country_query})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt=ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해 줘: {sentence}
                                        French sentence: (print from here)""")

model = ChatOpenAI(model="gpt-4o-mini")

output_parser=StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()}|prompt|model|output_parser
runnable_chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

In [ ]:
(RunnablePassthrough.assign(mult=lambda x:x["num"]*3)).invoke({"num": 3})

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x:x["num"]*3),
    modified=lambda x:x["num"]+1
)
runnable.invoke({"num":1})

In [19]:
def add_smile(x):
    return x+":)"

In [20]:
from langchain_core.runnables import RunnableLambda
add_smile = RunnableLambda(add_smile)

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt_str = "{topic}의 역사에 대해 세문장으로 설명해 주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model_name='gpt-4o-mini')
output_parser = StrOutputParser()
chain=prompt|model|output_parser

In [22]:
from langchain_core.runnables import RunnableLambda

def add_thank(x):
    return x+"들어주셔서 감사합니다:)"

add_thank = RunnableLambda(add_thank)

In [ ]:
chain=prompt|model|output_parser|add_thank
chain.invoke("반도체")

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x:x["num"]+1
)
runnable.invoke({"num": 1})

In [ ]:
runnable=RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thank
)
runnable.invoke("안녕하세요")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model='gpt-4o-mini', max_completion_tokens=128, temperature = 0)

history_prompt=ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요")
celeb_prompt=ChatPromptTemplate.from_template("{topic}분야의 유명인사 3명의 이름만 알려주세요")

output_parser = StrOutputParser()

history_chain=history_prompt|model|output_parser
celeb_chain=celeb_prompt|model|output_parser

map_chain=RunnableParallel(history=history_chain, celeb=celeb_chain)
result=map_chain.invoke({"topic": "AI"})
result